In [1]:
import pandas as pd

import camelot

import pdfplumber

import os

import re

import numpy as np

from PyPDF2 import PdfReader

In [2]:
def process_all_pdfs_in_folder(source_path, sink_path, desired_disease):
    year_df = []
    for filename in os.listdir(source_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(source_path, filename)
            print(f"Processando o arquivo: {pdf_path}")
            week_df = extract_and_process_tables(pdf_path, desired_disease, filename)
            year_df.append(week_df)
            
    if year_df:
        concatenated_df = pd.concat(year_df, ignore_index=True)

        pdf_filename = source_path.rsplit('/', 1)[-1]
        print("Salvando")
        csv_filename = os.path.splitext(pdf_filename)[0] + desired_disease + '.csv'
        output_path = os.path.join(sink_path, csv_filename)
        concatenated_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Arquivo salvo: {csv_filename}")
            

In [3]:
def extract_and_process_tables(pdf_path, desired_disease, filename):
    all_dfs = []
    pdf = pdfplumber.open(pdf_path)

    for page_num in range(len(pdf.pages)):
        try:
            tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

            if tables:
                df = tables[0].df
                page = pdf.pages[page_num]
                table = page.extract_table()
                df_header = pd.DataFrame(table)
                                
                if (not df_header.empty 
                    and df_header.shape[0] > 1  # Verifica se há pelo menos 2 linhas
                    and df_header.shape[1] > 1  # Verifica se há pelo menos 2 colunas
                    and df_header.iloc[0, 1] is not None
                    and df_header.iloc[0, 5] is not None 
                    and df_header.iloc[0, 1] is not None
                    and desired_disease in df_header.iloc[0, 5]):
                    
                    first = None
                    

                    for n in range(4, 15):
                        cell_value = dfx.iloc[n, 0]
                        if cell_value  == 'Aguascalientes':
                            first = n
                            break
                            
                    df_all = process_table(df, df_header, filename, first)
                    
                        
                    if not df_all.empty:
                        all_dfs.append(df_all)
#                         print(all_dfs)
                        print(f"Processando a página {page_num + 1} com a tabela desejada.")

        except Exception as e:
            print(f"Erro ao processar a página {page_num + 1}: {e}")

    pdf.close()

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    # Salva o DataFrame final em um arquivo CSV
#     pdf_filename = os.path.basename(pdf_path)
#     csv_filename = os.path.splitext(pdf_filename)[0] + '_filtered.csv'
#     output_path = os.path.join('/home/pirata/Documents/projects/epidemic_database/bases', csv_filename)
#     final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    return final_df

In [14]:
# Defina a função para processar as tabelas do PDF
def process_table(df, df_header, filename, first):
    
    
    if df.empty or df_header.empty:
        print("DataFrame ou cabeçalho vazio.")
        return pd.DataFrame()  # Retorna um DataFrame vazio se não houver dados
    
    weeknumber = re.search(r'-SE(\d+)\.pdf', filename).group(1)

    df = df.iloc[first:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[:]

    # df.replace('', np.nan, inplace=True)

    df = df.loc[:, (df != '').any(axis=0)]

    df.dropna(axis=1, how='all')

    col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
    df.columns = col_names

    for col in col_names[1:len(df.columns)]:
        df[col] = df[col].str.replace(' ', '')

    first_col_name = df.columns[1]
    df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]
    df = df[~df['Coluna1'].str.startswith(('TOTAL', '&','FUENTE','§FUENTE'))]


    indices_para_converter = range(1, len(df.columns))
    for idx in indices_para_converter:
        if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
            df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)

    num_col = len(df.columns)


    df1 = df.iloc[:, 0:5].copy()
    if len(df_header) > 1 and len(df_header.columns) > 1:
        df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
        df1['year'] = '2016'
    else:
        df1['disease'] = ''
        df1['year'] = ''


    df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)  

    df1['Acum_Year'] = '2015'
    df1['week'] = weeknumber


    novos_nomes = [
        'ENTIDAD FEDERATIVA',
        'Sem.',
        'M',
        'F',
        'Acum.',
    ]


    df1.columns = novos_nomes + list(df1.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 5, 6, 8, 9]
    df1 = df1.iloc[:, column_order]

    df2 = df.iloc[:, [0] + list(range(5, 9))].copy()

    if len(df_header) > 0 and len(df_header.columns) > 5:
        df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
        df2['year'] = '2016'
    else:
        df2['disease'] = ''
        df2['year'] = ''


    df2['MF'] = df2.get('Coluna7', 0) + df2.get('Coluna8', 0)

    df2['Acum_Year'] = '2015'

    df2['week'] = weeknumber




    df2.columns = novos_nomes + list(df2.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 5, 6, 8, 9]
    df2 = df2.iloc[:, column_order]

    
    df_all = pd.concat([df1, df2], axis=0, ignore_index=True)
    

    return df_all

In [20]:

source_path = '/home/pirata/Documents/projects/epidemic_database/files/2016/semanal'
sink_path = '/home/pirata/Documents/projects/epidemic_database/bases/2016/Chikungunya'
desired_disease = 'Chikungunya'
process_all_pdfs_in_folder(source_path, sink_path, desired_disease)

Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE39.pdf


/home/pirata/.local/lib/python3.12/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 40 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE16.pdf
Processando a página 40 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE20.pdf
Processando a página 40 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE48.pdf


/home/pirata/.local/lib/python3.12/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Erro ao processar a página 8: index 5 is out of bounds for axis 0 with size 3


...

Processando a página 44 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE26.pdf
Processando a página 40 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE6.pdf
Processando a página 44 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE42.pdf


/home/pirata/.local/lib/python3.12/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Erro ao processar a página 4: index 5 is out of bounds for axis 0 with size 4
Erro ao processar a página 5: index 5 is out of bounds for axis 0 with size 4
Erro ao processar a página 7: index 5 is out of bounds for axis 0 with size 2


/home/pirata/.local/lib/python3.12/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 44 com a tabela desejada.
Salvando
Arquivo salvo: semanalChikungunya.csv


In [4]:
filename = 'BOL-EPID-2016-SE33.pdf'

pdf_path = '/home/pirata/Documents/projects/epidemic_database/files/2016_teste/BOL-EPID-2016-SE33.pdf'

page_num = 43

pdf = pdfplumber.open(pdf_path)

tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

# if (not df_header.empty 
#                     and df_header.shape[0] > 1  # Verifica se há pelo menos 2 linhas
#                     and df_header.shape[1] > 1  # Verifica se há pelo menos 2 colunas
#                     and df_header.iloc[0, 1] is not None
#                     and desired_disease in df_header.iloc[0, 5]):
#     print('é')
    

if tables:
    dfx = tables[0].df
    page = pdf.pages[page_num]
    table = page.extract_table()
    df_header = pd.DataFrame(table)

In [5]:
first = None

for n in range(4, 15):
    cell_value = dfx.iloc[n, 0]
    if cell_value  == 'Aguascalientes':
        first = n
        break
        
        
print(n)

5


In [10]:

weeknumber = re.search(r'-SE(\d+)\.pdf', filename).group(1)

df = dfx.iloc[first:].reset_index(drop=True)
df.columns = df.iloc[0]
df = df[:]

# df.replace('', np.nan, inplace=True)

df = df.loc[:, (df != '').any(axis=0)]

df.dropna(axis=1, how='all')

col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
df.columns = col_names

for col in col_names[1:len(df.columns)]:
    df[col] = df[col].str.replace(' ', '')

first_col_name = df.columns[1]
df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]
df = df[~df['Coluna1'].str.startswith(('TOTAL', '&','FUENTE','§FUENTE'))]


indices_para_converter = range(1, len(df.columns))
for idx in indices_para_converter:
    if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
        df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)
        
num_col = len(df.columns)
        

df1 = df.iloc[:, 0:5].copy()
if len(df_header) > 1 and len(df_header.columns) > 1:
    df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
    df1['year'] = '2016'
else:
    df1['disease'] = ''
    df1['year'] = ''
    
    
df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)  

df1['Acum_Year'] = '2015'
df1['week'] = weeknumber


novos_nomes = [
    'ENTIDAD FEDERATIVA',
    'Sem.',
    'M',
    'F',
    'Acum.',
]


df1.columns = novos_nomes + list(df1.columns[5:])
column_order = [0, 1, 2, 3, 7, 4, 5, 6, 8, 9]
df1 = df1.iloc[:, column_order]

df2 = df.iloc[:, [0] + list(range(5, 9))].copy()

if len(df_header) > 0 and len(df_header.columns) > 5:
    df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
    df2['year'] = '2016'
else:
    df2['disease'] = ''
    df2['year'] = ''
    
    
df2['MF'] = df2.get('Coluna7', 0) + df2.get('Coluna8', 0)

df2['Acum_Year'] = '2015'

df2['week'] = weeknumber




df2.columns = novos_nomes + list(df2.columns[5:])
column_order = [0, 1, 2, 3, 7, 4, 5, 6, 8, 9]
df2 = df2.iloc[:, column_order]



df2

,ENTIDAD FEDERATIVA,Sem.,M,F,MF,Acum.,disease,year,Acum_Year,week
0,Aguascalientes,0,0,0,0,0,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
1,Baja California,0,0,0,0,0,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
2,Baja California Sur,2,13,10,23,1,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
3,Campeche,0,10,12,22,140,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
4,Coahuila,0,0,1,1,1,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
5,Colima,0,2,2,4,314,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
6,Chiapas,0,3,4,7,517,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
7,Chihuahua,0,0,0,0,0,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
8,Distrito Federal,0,0,0,0,0,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33
9,Durango,0,0,0,0,0,§Enfermedad por Virus Chikungunya CIE-10ª REV....,2016,2015,33


In [58]:
len(df.columns)

14

In [7]:
display(re.search(r'-SE(\d+)\.pdf', filename).group(1))

'51'

In [65]:
df_header.iloc[0, 4]

'Fiebre Manchada\nCIE-10ª REV.\nA77.0'

In [43]:
desired_disease = 'Leptospirosis'

desired_disease in df_header.iloc[0, 5]

True

In [62]:
len(dfx.columns)

16

In [6]:
df_header

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,ENTIDAD\nFEDERATIVA,Tracoma\nCIE-10ª REV.\nA71,None,None,None,§Enfermedad por Virus\nChikungunya\nCIE-10ª RE...,None,None,None,Leishmaniasis Visceral\nCIE-10ª REV.\nB55.0,None,None,None
1,None,2016,None,None,2015,2016,None,None,2015,2016,None,None,2015
2,None,Sem.,Acum.,None,Acum.,Sem.,Acum.,None,Acum.,Sem.,Acum.,None,Acum.
3,None,None,M,F,None,None,M,F,None,None,M,F,None
4,Aguascalientes,None,- -\n- -\n- -\n- -\n- -\n- -\n- -\n- -\n- -\n-...,None,-\n-\n-\n-\n-\n-\n2\n-\n-\n-\n-\n-\n-\n-\n-\n-...,-\n-\n2\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-...,- -\n- -\n13 10\n10 12\n- 1\n2 2\n3 4\n- -\n- ...,None,-\n-\n1\n140\n1\n314\n517\n-\n-\n-\n-\n1 383\n...,-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-...,- -\n- -\n- -\n- -\n- -\n- -\n- -\n- -\n- -\n-...,None,-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-\n-...
5,TOTAL -,None,- 1,None,2,18,156 257,None,5 315,-,- -,None,-


In [7]:
dfx

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,FEDERATIVA,,,,,,,,,,,,,,,,
1,,,2016,,,,2015,,2016,,2015,,2016,,,,2015
2,,,,Acum.,,,,,Acum.,,,,,Acum.,,,
3,,Sem.,,,,,Acum.,Sem.,,,Acum.,Sem.,,,,,Acum.
4,,,M,,F,,,,M,F,,,M,,F,,
5,Aguascalientes,-,,-,,-,-,-,-,-,-,-,,-,,-,-
6,Baja California,-,,-,,-,-,-,-,-,-,-,,-,,-,-
7,Baja California Sur,-,,-,,-,-,2,13,10,1,-,,-,,-,-
8,Campeche,-,,-,,-,-,-,10,12,140,-,,-,,-,-
9,Coahuila,-,,-,,-,-,-,-,1,1,-,,-,,-,-
